In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import pandas as pd
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset, random_split, TensorDataset
import matplotlib.pyplot as plt
import time
import os
from sklearn.model_selection import train_test_split
import spacy
import re

In [12]:
#Import Dataset
#For preprocessing purposes: Clean Text of special characters/URLs/HTML tags/punctuation and put in all lowercase,
#Split text into words or "tokens", remove "stopwords" such as "and", "is", or "the,"
#Reduce words to root or stem form through "lemmatization" or "stemming" to reduce complexity,
#Apply "vectorization" to convert word or token data into a set of numerical data, and
#Pad the data to normalize all the variables so they can be more easily inputted to a ML algorithm
SentimentData = pd.read_csv('data/CleanSentimentTwtData.csv')

2


In [ ]:
#Replaces @usernames with "name", Cleans text of HTMl tags, URLS, removes extra spaces, and makes the text lowercase
def clean_text(text):
    text = re.sub(r'@\w+', 'username', text)
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'http\S+', '', text)    
    text = re.sub(r'\s+', ' ', text).strip()
    return text.lower()

#Processes the other needed parts
nlp = spacy.load('en_core_web_sm')
def preprocess_text(dset):
    processed_text = []
    for text in dset['Text'].values: #loops through all values in 'Text' col
        result = []
        if nlp(text).lang_ == 'en': #if the data point is in english, proceed
            for token in nlp(text): #loops through each word or "token" in the datapoint
                if token.is_alpha and not token.is_stop: #ignores non-alphabetic tokens and "stopword" tokens
                    result.append(token.lemma_) #keeps the lemmatization of the token
            processed_text.append(' '.join(result)) #joins the tokens back together with a space between
    return processed_text

In [11]:
#Create new torch Dataset-inherited object and transform pandas dataset into it
class SentimentTwtData(Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        return self.data[index]
    
SentimentDset = SentimentTwtData(SentimentData)

In [19]:
#Split the data randomly, 80% train, 20% test
train_size = int(.8 * len(SentimentDset))
test_size = len(SentimentDset) - train_size
train_dset, test_dset = random_split(SentimentDset, [train_size, test_size])
print(len(train_dset))

1280000


In [20]:
#Turn data into DataLoaders
batch_size = 500
n_iters = 12800
num_epochs = int(n_iters / (len(train_dset) / batch_size))
train_loader = DataLoader(dataset=train_dset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dset, batch_size=batch_size, shuffle=False)

In [ ]:
class SentimentAnalysisModel(nn.Module):
    def __init__(self,)